<a href="https://colab.research.google.com/github/gc2321/LangChain-Python/blob/main/2_3_vectorStore_retrievers.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import os

In [2]:
!pip install --upgrade --quiet langchain-openai langchain-community
!pip install -quiet openai


Usage:   
  pip3 install [options] <requirement specifier> [package-index-options] ...
  pip3 install [options] -r <requirements file> [package-index-options] ...
  pip3 install [options] [-e] <vcs project url> ...
  pip3 install [options] [-e] <local project path> ...
  pip3 install [options] <archive url/path> ...

no such option: -u


In [3]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [4]:
open_ai_api ="/content/gdrive/MyDrive/llm/api_key.txt"
f = open(open_ai_api)
api_key = f.read()

In [5]:
os.environ['OPENAI_API_KEY'] = api_key

# Vector Store Retriever

In [6]:
!pip install chromadb

In [7]:
from langchain.vectorstores import Chroma
from langchain.document_loaders import WikipediaLoader
from langchain_openai import OpenAIEmbeddings
from langchain.text_splitter import CharacterTextSplitter

In [8]:
from langchain.document_loaders import TextLoader
path = "/content/gdrive/MyDrive/llm/langchain/some_data/FDR_State_of_Union_1944.txt"
loader = TextLoader(path)
documents = loader.load()

In [9]:
embedding_function = OpenAIEmbeddings()

In [10]:
# split it into chunks
text_splitter = CharacterTextSplitter.from_tiktoken_encoder(chunk_size=500)
docs = text_splitter.split_documents(documents)

In [39]:
!ls -l ./mk_ultra

ls: cannot access './mk_ultra': No such file or directory


In [12]:
db = Chroma.from_documents(docs, embedding_function,persist_directory='./speech_embedding_db')

In [13]:
db.persist()

/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:119: LangChainDeprecationWarning: Since Chroma 0.4.x the manual persistence method is no longer supported as docs are automatically persisted.
  warn_deprecated(


In [14]:
# reloading
db_connection = Chroma(persist_directory='./speech_embedding_db/',embedding_function=embedding_function)

In [15]:
docs = db.similarity_search('slavery')

In [16]:
docs[0].page_content

'It is our duty now to begin to lay the plans and determine the strategy for the winning of a lasting peace and the establishment of an American standard of living higher than ever before known. We cannot be content, no matter how high that general standard of living may be, if some fraction of our people—whether it be one-third or one-fifth or one-tenth- is ill-fed, ill-clothed, ill housed, and insecure.\n\nThis Republic had its beginning, and grew to its present strength, under the protection of certain inalienable political rights—among them the right of free speech, free press, free worship, trial by jury, freedom from unreasonable searches and seizures. They were our rights to life and liberty.\n\nAs our Nation has grown in size and stature, however—as our industrial economy expanded—these political rights proved inadequate to assure us equality in the pursuit of happiness.\n\nWe have come to a clear realization of the fact that true individual freedom cannot exist without economi

In [17]:
## assign to retriever
retriever = db_connection.as_retriever()

In [18]:
search_kwargs = {"score_threshold":0.8,"k":4}
docs = retriever.invoke("President",search_kwargs=search_kwargs)

In [19]:
print(len(docs))

4


In [20]:
docs[0].page_content

'I ask the Congress to explore the means for implementing this economic bill of rights- for it is definitely the responsibility of the Congress so to do. Many of these problems are already before committees of the Congress in the form of proposed legislation. I shall from time to time communicate with the Congress with respect to these and further proposals. In the event that no adequate program of progress is evolved, I am certain that the Nation will be conscious of the fact.\n\nOur fighting men abroad- and their families at home- expect such a program and have the right to insist upon it. It is to their demands that this Government should pay heed rather than to the whining demands of selfish pressure groups who seek to feather their nests while young Americans are dying.\n\nThe foreign policy that we have been following—the policy that guided us at Moscow, Cairo, and Teheran—is based on the common sense principle which was best expressed by Benjamin Franklin on July 4, 1776: "We mu

## MultiQuery Retriever

In [22]:
!pip install wikipedia

  Preparing metadata (setup.py) ... done
  Created wheel for wikipedia: filename=wikipedia-1.4.0-py3-none-any.whl size=11680 sha256=6a1c1d9d99bcae330a4170add877ce9701f53fc52750acfb04d4c5fa2ec8bec2
  Stored in directory: /root/.cache/pip/wheels/5e/b6/c5/93f3dec388ae76edc830cb42901bb0232504dfc0df02fc50de
Successfully built wikipedia


In [23]:
loader = WikipediaLoader(query='MKUltra')
documents = loader.load()

/usr/local/lib/python3.10/dist-packages/wikipedia/wikipedia.py:389: GuessedAtParserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("lxml"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 389 of the file /usr/local/lib/python3.10/dist-packages/wikipedia/wikipedia.py. To get rid of this warning, pass the additional argument 'features="lxml"' to the BeautifulSoup constructor.

  lis = BeautifulSoup(html).find_all('li')


In [24]:
len(documents)

24

## Split Documents

In [25]:
# split it into chunks
text_splitter = CharacterTextSplitter.from_tiktoken_encoder(chunk_size=500)
docs = text_splitter.split_documents(documents)

## OpenAI Connection for Embeddings

In [26]:
embedding_function = OpenAIEmbeddings()

## Embed Documents for ChromaDB

In [27]:
# load it into Chroma
db = Chroma.from_documents(docs, embedding_function,persist_directory='./mk_ultra')
db.persist()

## Use Chat Model to do Mulit Query

In [28]:
from langchain_openai import ChatOpenAI
from langchain.retrievers.multi_query import MultiQueryRetriever
question="When was this declassified?"
llm = ChatOpenAI(temperature=0)
retriever_from_llm = MultiQueryRetriever.from_llm(retriever=db.as_retriever(),llm=llm)

In [29]:
# Set logging for the queries
import logging
logging.basicConfig()
logging.getLogger('langchain.retrievers.multi_query').setLevel(logging.INFO)

In [30]:
# generate multiple queries and retrieve doc most similar and relevant to the generated queries
unique_docs = retriever_from_llm.invoke(input=question)

INFO:langchain.retrievers.multi_query:Generated queries: ['1. What is the date of the declassification of this information?', '2. Can you provide the specific time when this was no longer classified?', '3. When did this document become publicly available after being declassified?']


In [31]:
len(unique_docs)

8

In [32]:
print(unique_docs[0].page_content)

== Background ==
In 1974, a New York Times article was published that accused the CIA of illegal operations committed against US citizens. Authored by Seymour M. Hersh, it documented an intelligence operation against the anti-war movement, as well as "break-ins, wiretapping and the surreptitious inspection of mail" conducted since the 1950s. According to former CIA Official Cord Meyer, these disclosures "Convinced large sections of the American public that the CIA had become a domestic Gestapo and stimulated an overwhelming demand for the wide-ranging congressional investigations that were to follow."
Hersh had been tipped off to the possibility of an "in house operation" by an unidentified member of the CIA in spring of 1974. He embarked on an investigation, speaking to sources that included CIA Chief of Counterintelligence James Angleton. Although he was not aware of its existence, Hersh uncovered much information that had been documented in the "Family Jewels", a report ordered by D

# Context Compression

###### Use LLM call to extract relevant information from doc stored in the vector store

## Connected Embed Documents via ChromaDB

In [33]:
db_connection = Chroma(persist_directory='./mk_ultra',embedding_function=embedding_function)

## Contextual Compression

In [34]:
from langchain_openai import ChatOpenAI
from langchain.retrievers import ContextualCompressionRetriever
from langchain.retrievers.document_compressors import LLMChainExtractor

In [35]:
llm = ChatOpenAI(temperature=0)
compressor = LLMChainExtractor.from_llm(llm)

In [36]:
compression_retriever = ContextualCompressionRetriever(base_compressor=compressor, base_retriever=db_connection.as_retriever())

In [37]:
docs = db_connection.similarity_search('When was this declassified?')

In [38]:
docs[0]

Document(page_content='== Background ==\nIn 1974, a New York Times article was published that accused the CIA of illegal operations committed against US citizens. Authored by Seymour M. Hersh, it documented an intelligence operation against the anti-war movement, as well as "break-ins, wiretapping and the surreptitious inspection of mail" conducted since the 1950s. According to former CIA Official Cord Meyer, these disclosures "Convinced large sections of the American public that the CIA had become a domestic Gestapo and stimulated an overwhelming demand for the wide-ranging congressional investigations that were to follow."\nHersh had been tipped off to the possibility of an "in house operation" by an unidentified member of the CIA in spring of 1974. He embarked on an investigation, speaking to sources that included CIA Chief of Counterintelligence James Angleton. Although he was not aware of its existence, Hersh uncovered much information that had been documented in the "Family Jewel

In [39]:
compressed_docs = compression_retriever.invoke("When was this declassified?")

In [40]:
compressed_docs[0].page_content

'In 1974, a New York Times article was published that accused the CIA of illegal operations committed against US citizens. Authored by Seymour M. Hersh, it documented an intelligence operation against the anti-war movement, as well as "break-ins, wiretapping and the surreptitious inspection of mail" conducted since the 1950s. According to former CIA Official Cord Meyer, these disclosures "Convinced large sections of the American public that the CIA had become a domestic Gestapo and stimulated an overwhelming demand for the wide-ranging congressional investigations that were to follow."'